In [1]:
import semantic_kernel as sk
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
print(os.getenv("OPENAI_API_BASE"))


https://alkopenai2.openai.azure.com/


In [2]:
# for this example I want a valid logger
import logging

# Create a logger
logger = logging.getLogger("SK")

# Set the logging level to INFO
logger.setLevel(logging.DEBUG)

# Create a console handler
console_handler = logging.StreamHandler()

# Set the level of the console handler to INFO
console_handler.setLevel(logging.DEBUG)

# Create a formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Set the formatter for the console handler
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)


In [3]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
)

model = os.getenv("AZURE_GPT4_MODEL", "gpt4")
endpoint = os.getenv("OPENAI_API_BASE")
kernel = sk.Kernel(log=logger)
kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        model,
        endpoint = endpoint,
        api_key = os.getenv("OPENAI_API_KEY")
    ),
)

print(f"Using deploy {endpoint} with model {model}")

SyntaxError: f-string: unmatched '(' (2166899585.py, line 16)

In [ ]:
# Now we need to import the plugin
from plugins.AudioVideoPlugin.AudioVideo import AudioVideo

# Now you can import the plugin importing skill directly from the function you declared
# in the plugin directory. The import_skill does not need the path, it only need an
# instance of the skill and the name of the skill
extractaudio_plugin = kernel.import_skill(AudioVideo(), skill_name="AudioVideoPlugin")

plugins_directory = "./plugins"

# Import the OrchestratorPlugin from the plugins directory.
publishing_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "PublishingPlugin"
)


In [ ]:
from pprint import pprint
# want to print all the keys of extractaudio_plugin that is a dictionary

pprint (extractaudio_plugin["ExtractAudio"])
pprint (extractaudio_plugin["TranscriptTimeline"])
pprint (publishing_plugin["VideoTimelineCreator"])

In [ ]:
# you can verify if cuda is available.
import torch
torch.cuda.is_available()

In [ ]:
# Now we can use the skill
# Run the Sqrt function with the context.
result = await kernel.run_async(
    extractaudio_plugin["ExtractAudio"],
    extractaudio_plugin["TranscriptTimeline"],
    publishing_plugin["VideoTimelineCreator"],
    input_str="""S:\\OneDrive\\Youtube\\AI\\SemanticChain\\MontaggiCompleti\\200-intro-kernel-memory.mp4"""
)


In [ ]:
print(result)